In [15]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

In [16]:
import numpy
import tflearn
import tensorflow
import random
import json
tensorflow.__version__

'1.15.0'

In [17]:
with open("intents.json") as file:
    data = json.load(file)

words = []
labels = [] # TAG
docs_x = [] # Frase de entrada
docs_y = [] # TAG Resultado

for intent in data["intents"]:
    for pattern in intent["patterns"]:
        wrds = nltk.word_tokenize(pattern) # devuelve una lista con las diferentes palabras
        words.extend(wrds)                 # extiende la lista de words, haciendo que sea solo una lista
        docs_x.append( wrds )
        docs_y.append( intent["tag"] )
        
    if intent["tag"] not in labels:
        labels.append(intent["tag"])

words = [ stemmer.stem( w.lower() ) for w in words if w != "?"]
words = sorted ( list( set( words ) ) )

labels = sorted( labels )

In [18]:
print(words)
print("==============================")
print(labels[:12])
print("==============================")
print(docs_x[:12])
print("==============================")
print(docs_y)
print("==============================")

['!', 'acc', 'adio', 'ah', 'ahor', 'alguy', 'animac', 'animada', 'asdfass', 'asdfsdf', 'aventur', 'ayudam', 'ayudarm', 'aã±o', 'aã±os', 'buen', 'bye', 'chao', 'chist', 'cienc', 'comed', 'como', 'cont', 'crim', 'cual', 'cuanto', 'cuentam', 'de', 'debo', 'dia', 'diseã±ado', 'dormir', 'dram', 'duerm', 'edad', 'en', 'er', 'es', 'escribir', 'esta', 'fantas', 'ficc', 'fuist', 'hac', 'hago', 'has', 'hasm', 'hay', 'hecho', 'hol', 'hum', 'infantil', 'ir', 'las', 'listado', 'llam', 'llama', 'llamart', 'luego', 'mam', 'me', 'mi', 'misterio', 'muestram', 'no', 'nombr', 'nueva', 'objetivo', 'par', 'pas', 'pel', 'pelicula', 'por', 'porqu', 'pued', 'pusieron', 'que', 'qui', 'quiero', 'reir', 'rom', 'sab', 'sdfsadfasdfasdf', 'se', 'skynet', 'sobr', 'su', 'suspenso', 'tal', 'te', 'ten', 'tenga', 'tengo', 'ter', 'tien', 'toda', 'tu', 'un', 'veo', 'voy', 'y']
['accion', 'adios', 'animacion', 'aventura', 'chiste', 'ciencia ficcion', 'comedia', 'crimen', 'drama', 'edad', 'fail', 'fantasia']
[['Hola', '!'],

In [5]:
labels

['accion',
 'adios',
 'animacion',
 'aventura',
 'chiste',
 'ciencia ficcion',
 'comedia',
 'crimen',
 'drama',
 'edad',
 'fail',
 'fantasia',
 'help',
 'horas',
 'infantil',
 'listado',
 'misterio',
 'nombre',
 'porque_ese_nombre',
 'romance',
 'saludo',
 'suspenso',
 'terror']

In [6]:
# creando numeros que representen a las palabras
training = []
output = []

out_empty = [ 0 for _ in range (len(labels)) ]
print (out_empty)

for x, doc in enumerate( docs_x ):
    bag = []
    wrds = [ stemmer.stem(w) for w in doc ]
    
    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)
    
    output_row = out_empty[:]
    output_row [ labels.index( docs_y[x] ) ]  = 1
    
    training.append( bag )
    output.append( output_row )
    
    
training = numpy.array( training )
output = numpy.array( output )

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [7]:
print (docs_x[0])
print(training[0])

['Hola', '!']
[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [8]:
print (labels)
print(output[0])

['accion', 'adios', 'animacion', 'aventura', 'chiste', 'ciencia ficcion', 'comedia', 'crimen', 'drama', 'edad', 'fail', 'fantasia', 'help', 'horas', 'infantil', 'listado', 'misterio', 'nombre', 'porque_ese_nombre', 'romance', 'saludo', 'suspenso', 'terror']
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0]


In [19]:
# para eliminar todos los ajustes anteriores para restablecer , define un grapho vacio
tensorflow.reset_default_graph()

net = tflearn.input_data( shape=[None,len( training[0] )] )
net = tflearn.fully_connected(net,280)
net = tflearn.fully_connected(net,200)
net = tflearn.fully_connected(net,300)
net = tflearn.fully_connected(net,90)
net = tflearn.fully_connected(net,len(output[0]), activation = "softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)


try:
    model.load("save_model/model.tflearn")
    print ("recargado!")
except:
    model.fit(training, output, n_epoch = 500 , batch_size = 8, show_metric = True)
    model.save("save_model/model.tflearn")

# model.fit(training, output, n_epoch = 100 , batch_size = 8, show_metric = True)
# model.save("save_model/model.tflearn")

INFO:tensorflow:Restoring parameters from C:\Users\Toshiba\PROYECTOS\#ProcesamientoLenguajeNatural\project_chatbot\save_model\model.tflearn
recargado!


In [20]:
def bag_of_words (s, words):
    bag = [0 for _ in range ( len(words) )]
    
    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem( word.lower() ) for word in s_words ]
    
    for se  in s_words:
        for i,w in enumerate (words):
            if w == se:
                bag[i] = 1
                
    return numpy.array(bag)

In [12]:
def verificar_peliculas(tag):
    peliculas = json.load(open('peliculas.json', 'r', encoding='utf-8'))
    
    if tag == "listado":
        for i in peliculas["peliculas"]:
            print(i["year"]," ",i["pelicula"])
            
    if tag in labels:
        # if para buscar por genero
        if tag not in peliculas['peliculas']:
            
            for datos in peliculas["peliculas"]:
                for gen in datos['genero']:
                    if tag.lower() == gen.lower():
                        print(gen," ", datos["pelicula"])

In [21]:
verificar_peliculas("drama")

Drama   Historia de un matrimonio
Drama   Estafadoras de Wall Street
Drama   It capítulo dos
drama   El jilguero
Drama   En la hierba alta
Drama   Cada día
Drama   El Rey Arturo: La Leyenda de la Espada
Drama   Estación Zombie
Drama   La llegada
Drama   La leyenda de Tarzán


In [25]:
while True:
    inp = input (" >>> you: ")
    
    resp = model.predict([ bag_of_words(inp,words) ])[0] # retorna la primera linea y solo es una simple lista
#     print (resp)
    resp_index = numpy.argmax(resp)
#     print (resp [resp_index])
#     print (resp_index)
    
    if resp [resp_index] > 0.70:
        tag = labels[resp_index]
    else:
        tag = "fail"
        
    for tg in data["intents"]:
        if tg["tag"] == tag:
            responses = tg["responses"]

#     print (tag)
#     print(responses)
    print ( " === IA: ",random.choice(responses) )
    verificar_peliculas(tag)
    if tag == "adios":
        break

 >>> you: asefasfd
 === IA:  no se nada de eso, pregunta otra cosa
 >>> you: etrhhrtnfg
 === IA:  podrias escribir mas claro?
 >>> you: q u iero peliculas de terror
 === IA:  genial, aqui tengo todas las peliculas de tipo terror
Terror   It capítulo dos
Terror   Boda sangrienta
Terror   En la hierba alta
Terror   El despertar de los muertos vivientes
Terror   Feliz día de tu muerte
Terror   Life: Vida Inteligente
Terror   Estación Zombie
 >>> you: adios
 === IA:  hasta la vista Baby!
